# Tutorial: Make an xarray Dataset CF-1.12 compliant

This notebook shows how to use `netcdf-cf-coercer` to:
- inspect CF compliance with `ds.cf.check_compliant()`
- apply safe fixes with `ds.cf.make_compliant()`

The object stays an `xarray.Dataset` throughout.


## 1. Setup


In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from __future__ import annotations

import numpy as np
import xarray as xr
import rich

import netcdf_cf_coercer  # noqa: F401


## 2. Build a dataset with missing CF metadata


In [15]:
ds = xr.Dataset(
    data_vars={
        "temperature": (("time", "lat", "lon"), np.random.rand(2, 3, 4)),
    },
    coords={
        "time": [0, 1],
        "lat": [10.0, 20.0, 30.0],
        "lon": [100.0, 110.0, 120.0, 130.0],
    },
)

ds


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8052 0.5414 ... 0.6556 0.5747

## 3. Check compliance before coercion


In [30]:
issues_before = ds.cf.check_compliant(domain='borland', pretty_print=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ CF Compliance Report                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CF version     CF-1.12               
Engine         cfchecker             
Engine status  ok                    
Counts         fatal=0 error=0 warn=8

                          Global Findings                          
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope  ┃ Severity ┃ Detail                                      ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ global │ WARN     │ (2.6.1): No 'Conventions' attribute present │
└────────┴──────────┴─────────────────────────────────────────────┘

                             Coordinate Findings                             
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope ┃ Severity ┃ Detail                                                 ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ lat   │ WARN     │ (3): No standard_name or long_name attribute specified │
│ lat   │ WARN     │ (3.1): units attribute should be present               │
│ lon   │ WARN     │ (3): No standard_name or long_name attribute specified │
│ lon   │ WARN     │ (3.1): units attribute should be present               │
│ time  │ WARN     │ (3): No standard_name or long_name attribute specified │
│ time  │ WARN     │ (3.1): units attribute should be present               │
└───────┴──────────┴────────────────────────────────────────────────────────┘

                                 Variable Findings                                 
┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Scope       ┃ Severity ┃ Detail                                                 ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ temperature │ WARN     │ (3): No standard_name or long_name attribute specified │
└─────────────┴──────────┴────────────────────────────────────────────────────────┘

                                              Suggested Improvements                                               
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Variable    ┃ Suggestion                                                                                        ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ temperature │ {'recommended_standard_names': ['air_temperature', 'brightness_temperature',                      │
│             │ 'canopy_temperature'], 'recommended_units': 'K', 'domain': 'borland'}                             │
└─────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────┘

## 4. Apply compliance fixes


In [31]:
fixed = ds.cf.make_compliant()
fixed['temperature'].attrs['standard_name'] = 'sea_surface_temperature'
fixed['temperature'].attrs['units'] = 'degC'

print(type(fixed))
print("Conventions:", fixed.attrs.get("Conventions"))
print("lat attrs:", fixed["lat"].attrs)
print("lon attrs:", fixed["lon"].attrs)
print("time attrs:", fixed["time"].attrs)

fixed


<class 'xarray.core.dataset.Dataset'>
Conventions: CF-1.12
lat attrs: {'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_north', 'axis': 'Y'}
lon attrs: {'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_east', 'axis': 'X'}
time attrs: {'standard_name': 'time', 'axis': 'T'}


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8052 0.5414 ... 0.6556 0.5747
Attributes:
    Conventions:  CF-1.12

## 5. Re-check after coercion


In [32]:
issues_after = fixed.cf.check_compliant(pretty_print=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ CF Compliance Report                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CF version     CF-1.12               
Engine         cfchecker             
Engine status  unavailable           
Counts         fatal=0 error=0 warn=0

          Coordinate Findings          
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Scope ┃ Severity ┃ Detail           ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ time  │ INFO     │ coord_attr:units │
└───────┴──────────┴──────────────────┘

                                         Suggested Improvements                                          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Variable    ┃ Suggestion                                                                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ temperature │ {'units_check': {'status': 'mismatch', 'current_units': 'degC', 'expected_units': 'K'}} │
└─────────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                             Notes                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Note                                                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ cfchecker could not run; returned heuristic checks instead. │
└─────────────────────────────────────────────────────────────┘

╭───────────────────────────────────────────────── Checker Error ─────────────────────────────────────────────────╮
│ {'type': 'AttributeError', 'message': 'NetCDF: Attribute not found'}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 6. Other dimension patterns


In [33]:
# time-only
only_time = xr.Dataset(
    data_vars={"signal": (("time",), [1.0, 2.0, 3.0])},
    coords={"time": [0, 1, 2]},
)

# lat-lon only
only_space = xr.Dataset(
    data_vars={"field": (("lat", "lon"), np.ones((2, 2)))},
    coords={"lat": [0.0, 1.0], "lon": [10.0, 11.0]},
)

print("time-only conventions:", only_time.cf.make_compliant().attrs.get("Conventions"))
print("space-only conventions:", only_space.cf.make_compliant().attrs.get("Conventions"))


time-only conventions: CF-1.12
space-only conventions: CF-1.12


## 7. Write to NetCDF (optional)

Once compliant, you can write normally:

```python
fixed.to_netcdf("example_cf.nc")
```
